In [1]:
source = 'react'

In [2]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from IPython.display import display_markdown

In [3]:
chroma_client = chromadb.Client(Settings(
    chroma_db_impl='duckdb+parquet',
    persist_directory="./db" # Optional, defaults to .chromadb/ in the current directory
    )
)

Using embedded DuckDB with persistence: data will be stored in: ./db/source_react


In [4]:
ef = embedding_functions.InstructorEmbeddingFunction(
model_name="hkunlp/instructor-large", device="cuda")

/home/lucas/fluff/markdown-idx/python-api/.venv/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


In [5]:
collection = chroma_client.get_collection(name=source, embedding_function=ef)


In [6]:
# collection.create_index()

In [7]:
print(collection.count())

100


In [8]:
results = collection.query(
    query_texts=["When is data really deleted in Clickhouse with the new `DELETE` query"],
    n_results=10
)

In [9]:
import json
with open(f"../md-to-prompt/{source}.json", "r") as f:
    corpus = json.loads(f.read())

In [10]:
print(len(corpus))

4970


In [11]:
print(len(results['ids'][0]))

10


In [12]:
for a,b in zip(results['ids'][0], results['distances'][0]):
    idx = int(a)
    print(idx)
    print(" > ".join(corpus[idx]["headings"]))
    print(b)
    print()
    display_markdown(corpus[idx]["content"], raw=True)
    print()
    print()
    print()

41
docs/react/learn/extracting-state-logic-into-a-reducer.md > Consolidate state logic with a reducer {/consolidate-state-logic-with-a-reducer/}
0.3646610975265503



```js TaskList.js hidden
import { useState } from 'react';

export default function TaskList({tasks, onChangeTask, onDeleteTask}) {
  return (
    <ul>
      {tasks.map((task) => (
        <li key={task.id}>
          <Task task={task} onChange={onChangeTask} onDelete={onDeleteTask} />
        </li>
      ))}
    </ul>
  );
}

function Task({task, onChange, onDelete}) {
  const [isEditing, setIsEditing] = useState(false);
  let taskContent;
  if (isEditing) {
    taskContent = (
      <>
        <input
          value={task.text}
          onChange={(e) => {
            onChange({
              ...task,
              text: e.target.value,
            });
          }}
        />
        <button onClick={() => setIsEditing(false)}>Save</button>
      </>
    );
  } else {
    taskContent = (
      <>
        {task.text}
        <button onClick={() => setIsEditing(true)}>Edit</button>
      </>
    );
  }
  return (
    <label>
      <input
        type="checkbox"
        checked={task.done}
        onChange={(e) => {
          onChange({
            ...task,
            done: e.target.checked,
          });
        }}
      />
      {taskContent}
      <button onClick={() => onDelete(task.id)}>Delete</button>
    </label>
  );
}
```





80
docs/react/learn/extracting-state-logic-into-a-reducer.md > Writing concise reducers with Immer {/writing-concise-reducers-with-immer/}
0.36896246671676636



```js TaskList.js hidden
import { useState } from 'react';

export default function TaskList({tasks, onChangeTask, onDeleteTask}) {
  return (
    <ul>
      {tasks.map((task) => (
        <li key={task.id}>
          <Task task={task} onChange={onChangeTask} onDelete={onDeleteTask} />
        </li>
      ))}
    </ul>
  );
}

function Task({task, onChange, onDelete}) {
  const [isEditing, setIsEditing] = useState(false);
  let taskContent;
  if (isEditing) {
    taskContent = (
      <>
        <input
          value={task.text}
          onChange={(e) => {
            onChange({
              ...task,
              text: e.target.value,
            });
          }}
        />
        <button onClick={() => setIsEditing(false)}>Save</button>
      </>
    );
  } else {
    taskContent = (
      <>
        {task.text}
        <button onClick={() => setIsEditing(true)}>Edit</button>
      </>
    );
  }
  return (
    <label>
      <input
        type="checkbox"
        checked={task.done}
        onChange={(e) => {
          onChange({
            ...task,
            done: e.target.checked,
          });
        }}
      />
      {taskContent}
      <button onClick={() => onDelete(task.id)}>Delete</button>
    </label>
  );
}
```





76
docs/react/learn/extracting-state-logic-into-a-reducer.md > Writing reducers well {/writing-reducers-well/}
0.3841404318809509



...

```js tasksReducer.js
export default function tasksReducer(tasks, action) {
  switch (action.type) {
    case 'added': {
      return [
        ...tasks,
        {
          id: action.id,
          text: action.text,
          done: false,
        },
      ];
    }
    case 'changed': {
      return tasks.map((t) => {
        if (t.id === action.task.id) {
          return action.task;
        } else {
          return t;
        }
      });
    }
    case 'deleted': {
      return tasks.filter((t) => t.id !== action.id);
    }
    default: {
      throw Error('Unknown action: ' + action.type);
    }
  }
}
```





78
docs/react/learn/extracting-state-logic-into-a-reducer.md > Writing concise reducers with Immer {/writing-concise-reducers-with-immer/}
0.3988442122936249



```js App.js
import { useImmerReducer } from 'use-immer';
import AddTask from './AddTask.js';
import TaskList from './TaskList.js';

function tasksReducer(draft, action) {
  switch (action.type) {
    case 'added': {
      draft.push({
        id: action.id,
        text: action.text,
        done: false,
      });
      break;
    }
    case 'changed': {
      const index = draft.findIndex((t) => t.id === action.task.id);
      draft[index] = action.task;
      break;
    }
    case 'deleted': {
      return draft.filter((t) => t.id !== action.id);
    }
    default: {
      throw Error('Unknown action: ' + action.type);
    }
  }
}

export default function TaskApp() {
  const [tasks, dispatch] = useImmerReducer(tasksReducer, initialTasks);

  function handleAddTask(text) {
    dispatch({
      type: 'added',
      id: nextId++,
      text: text,
    });
  }

  function handleChangeTask(task) {
    dispatch({
      type: 'changed',
      task: task,
    });
  }

  function handleDeleteTask(taskId) {
    dispatch({
      type: 'deleted',
      id: taskId,
    });
  }

  return (
    <>
      <h1>Prague itinerary</h1>
      <AddTask onAddTask={handleAddTask} />
      <TaskList
        tasks={tasks}
        onChangeTask={handleChangeTask}
        onDeleteTask={handleDeleteTask}
      />
    </>
  );
}

let nextId = 3;
const initialTasks = [
  {id: 0, text: 'Visit Kafka Museum', done: true},
  {id: 1, text: 'Watch a puppet show', done: false},
  {id: 2, text: 'Lennon Wall pic', done: false},
];
```





89
docs/react/learn/extracting-state-logic-into-a-reducer.md > Writing concise reducers with Immer {/writing-concise-reducers-with-immer/} > Dispatch actions from event handlers {/dispatch-actions-from-event-handlers/}
0.40159347653388977



```js ContactList.js
export default function ContactList({contacts, selectedId, dispatch}) {
  return (
    <section className="contact-list">
      <ul>
        {contacts.map((contact) => (
          <li key={contact.id}>
            <button
              onClick={() => {
                // TODO: dispatch changed_selection
              }}>
              {selectedId === contact.id ? <b>{contact.name}</b> : contact.name}
            </button>
          </li>
        ))}
      </ul>
    </section>
  );
}
```


In [13]:
chroma_client.persist()

True